## 종속성 설치

In [ ]:
!pip install --upgrade --quiet  langchain langchain-core langchain-community langchain-text-splitters langchain-milvus langchain-openai bs4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 38.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 296.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 55.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 87.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 MB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 5.0 MB/s eta 0:00:00


In [ ]:
!pip install --upgrade--quiet tokenizers


Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: --upgrade--quiet


In [ ]:
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 32.2 MB/s eta 0:00:00


## LLM 모델 준비

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# 1. 모델 ID 설정
base_model_id = 'Bllossom/llama-3.2-Korean-Bllossom-3B'

# 2. BitsAndBytesConfig 설정
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, # 4bit quantization으로 모델을 불러와 메모리 소모를 줄임
    bnb_4bit_use_double_quant=False, # True로 설정한다면, 이중 양자화를 통해 메모리를 극도로 절약할 수 있음
    bnb_4bit_quant_type="nf4", # 기존 fp4 보다 고급 형식의 normal float 4-bit
    bnb_4bit_compute_dtype=torch.bfloat16 # 계산에 사용할 데이터 타입
)

# 3. 모델 로드
model = AutoModelForCausalLM.from_pretrained(base_model_id, quantization_config=bnb_config)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/904 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/180 [00:00<?, ?B/s]

In [ ]:
# 4. 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    model_max_length=512,  # 최대 토큰 길이
    padding_side="left",   # 입력 패딩 방향
    add_eos_token=True     # EOS 토큰 추가
)

tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/54.7k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [ ]:
from transformers import pipeline
from langchain.llms import HuggingFacePipeline
# LLM 파이프라인 설정 (LangChain과 통합)
llm_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=150
)
llm = HuggingFacePipeline(pipeline=llm_pipeline)

Device set to use cuda:0
<ipython-input-6-7a6f75dfb791>:10: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=llm_pipeline)


## 데이터 준비

<br>
Langchain csvLoader를 사용해 문서를 로드.

In [ ]:
from google.colab import drive

# 구글 드라이브 마운트
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
file_path = '/content/drive/My Drive/Colab Notebooks/RAG_MilVus/서지.csv'

In [ ]:
import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document

# CSV 파일 로드
file_path = '/content/drive/My Drive/Colab Notebooks/RAG_MilVus/서지.csv'
data = pd.read_csv(file_path)

# Replace NaN values in '책 소개' with an empty string
data['책 소개'] = data['책 소개'].fillna('')

# CSV 데이터를 LangChain Document로 변환
# Change metadata keys to start with an underscore and use English or abbreviations
documents = [
    Document(
        page_content=row['책 소개'],  # 책 소개를 주요 내용으로 설정
        metadata={
            "title": row['제목'],       # Changed to 'title'
            "author": row['저자'],       # Changed to 'author'
            "publisher": row['출판사'],   # Changed to 'publisher'
            "isbn": row['ISBN']       # Changed to 'isbn'
        }
    )
    for _, row in data.iterrows()
]

# 텍스트 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)
docs = text_splitter.split_documents(documents)

# 첫 번째 분할된 문서 출력
print(docs[1])

page_content='<전염병의 세계사>와 <전쟁의 세계사>로 국내 독자들에게 알려진 바 있으며, 현존하는 가장 탁월한 역사가 중 한명으로 꼽히는 윌리엄 맥닐 시카고대 역사학과 교수의 책. 해외에서입니다.' metadata={'title': '세계의 역사', 'author': '윌리엄 맥닐', 'publisher': '이산', 'isbn': 271590}


## 임베딩 모델 활용 데이터 임베딩 이후, Milvus 벡터 DB에 저장

<br>

문서로 Milvus 벡터 스토어를 초기화하여 Milvus 벡터 스토어에 문서를 로드하고 내부적으로 인덱스를 구축.

In [ ]:
from langchain_milvus import Milvus
from langchain.embeddings import HuggingFaceEmbeddings

# 임베딩 모델 설정
embedding_model = HuggingFaceEmbeddings(model_name="jhgan/ko-sroberta-multitask",model_kwargs = {'device': 'cuda:0'})

vectorstore = Milvus.from_documents(
    documents=docs,
    embedding=embedding_model,
    connection_args={
        "uri": "./milvus_demo.db",
    },
    drop_old=True,  # Drop the old Milvus collection if it exists
)

<ipython-input-10-b50d9655b078>:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="jhgan/ko-sroberta-multitask",model_kwargs = {'device': 'cuda:0'})


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.86k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2025-01-10 01:59:00,738 [ERROR][handler]: RPC error: [create_index], <MilvusException: (code=65535, message=invalid index type: HNSW, local mode only support FLAT IVF_FLAT AUTOINDEX: )>, <Time:{'RPC start': '2025-01-10 01:59:00.736598', 'RPC error': '2025-01-10 01:59:00.738094'}> (decorators.py:140)


### "BM-K/KoSimCSE-bert-multitask" 임베딩 모델 테스트
테스트 쿼리 문제를 사용하여 Milvus 벡터 스토어에서 문서를 검색합니다. 상위 2개 문서를 살펴보기.

In [ ]:
query = "거시경제학"
vectorstore.similarity_search(query, k=2)

[Document(metadata={'author': 'Frederic S. Mishkin', 'isbn': 995171, 'pk': 455190055181617317, 'publisher': 'Pearson Education Ltd', 'title': 'The Economics of Money, Banking and Financial Markets'}, page_content='For courses in Money and Banking or General Economics.\n\nAn Analytical Framework for Understanding입니다.'),
 Document(metadata={'author': '장하준', 'isbn': 941111, 'pk': 455190055181617365, 'publisher': '부키', 'title': '장하준의 경제학 강의'}, page_content='영국 케임브리지대 경제학과 교수인 장하준이 쓴 ‘일반인을 위한 경제학 입문서’. 경제란 무엇이고, 경제학을 왜 알아야 하는지에서 출발해 자본주의 경제가 어떤 과정을 통해 현재에입니다.')]

### jhgan/ko-sroberta-multitask 임베딩 모델 테스트
테스트 쿼리 문제를 사용하여 Milvus 벡터 스토어에서 문서를 검색합니다. 상위 2개 문서를 살펴보기.

In [ ]:
query = "거시경제학"
vectorstore.similarity_search(query, k=2)

[Document(metadata={'author': '정운찬, 김영식', 'isbn': 599922, 'pk': 455206329733087339, 'publisher': '율곡출판사', 'title': '거시경제론'}, page_content='이 책은 거시경제론을 다룬 개론서입니다. 거시경제론의 기초적이고 전반적인 내용을 학습할 수 있도록 구성했습니다.'),
 Document(metadata={'author': 'Andrew B. Abel, Ben S. Bernanke, Dean Croushore', 'isbn': 514313, 'pk': 455206329733087337, 'publisher': '지필미디어', 'title': '거시경제학'}, page_content='『거시경제학』은 경제학을 공부하는 학생들을 위해 고전학파와 케인즈파의 여러 가정을 명료하게 설명하는 단일 경제모형을 바탕으로 하나로 통일된 접근방법을 제시한 책이다. 특히 세계입니다.')]

## RAG 체인 구축하기
LCEL(LangChain 표현 언어)을 사용하여 RAG 체인을 구축합니다.

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# LangChain Prompt 설정
PROMPT_TEMPLATE = """
Human: 당신은 서지 정보를 분석하고 책을 추천하는 AI 비서입니다.
아래 제공된 서지 정보와 <question> 태그 안에 있는 사용자 질문을 기반으로 정확하고 간결한 답변을 제시하세요.
만약 질문에 대한 답을 알 수 없다면, "모르겠습니다."라고만 답하세요. 추측하지 마세요.

<서지정보>
{context}
</서지정보>

<질문>
{question}
</질문>

응답은 간결하며, 사실 기반이어야 하고 가능한 경우 데이터나 통계를 포함하세요.

Assistant:"""

prompt = PromptTemplate(
    template=PROMPT_TEMPLATE,
    input_variables=["context"]
)

# Convert the vector store to a retriever
retriever = vectorstore.as_retriever(k=3)


# Define a function to format the retrieved documents
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [ ]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)


res = rag_chain.invoke(query)
res

'\nHuman: 당신은 서지 정보를 분석하고 책을 추천하는 AI 비서입니다.\n아래 제공된 서지 정보와 <question> 태그 안에 있는 사용자 질문을 기반으로 정확하고 간결한 답변을 제시하세요.\n만약 질문에 대한 답을 알 수 없다면, "모르겠습니다."라고만 답하세요. 추측하지 마세요.\n\n<서지정보>\n이 책은 거시경제론을 다룬 개론서입니다. 거시경제론의 기초적이고 전반적인 내용을 학습할 수 있도록 구성했습니다.\n\n『거시경제학』은 경제학을 공부하는 학생들을 위해 고전학파와 케인즈파의 여러 가정을 명료하게 설명하는 단일 경제모형을 바탕으로 하나로 통일된 접근방법을 제시한 책이다. 특히 세계입니다.\n\nFor courses in Money and Banking or General Economics.\n\nAn Analytical Framework for Understanding입니다.\n\n연세대학교 경제학부 교수 서승환의 『미시경제학』. 문장을 다듬어 최대한 평이하게 읽도록 직관적 설명으로 구성한 미시경제학 교재다. 행태경제학과 관련된 이론까지 다루고 있다\n</서지정보>\n\n<질문>\n거시경제학\n</질문>\n\n응답은 간결하며, 사실 기반이어야 하고 가능한 경우 데이터나 통계를 포함하세요.\n\nAssistant: 거시경제학은 거시경제론의 기초적이고 전반적인 내용을 다룬 개론서입니다. 거시경제론의 기초적이고 전반적인 내용을 학습할 수 있도록 구성했습니다. 거시경제학을 공부하는 학생들을 위해 고전학파와 케인즈파의 여러 가정을 명료하게 설명하는 단일 경제모형을 바탕으로 하나로 통일된 접근방법을 제시한 책입니다. 특히 세계입니다. 이 책은 거시경제학을 이해하기 위한 기초적인 지식과 방법을 제공합니다. 거시경제학의 기초적이고 전반적인 내용을 학습할 수 있도록 구성된 책'

## Milvus 유사도 검색방식

In [ ]:
query = "장기적 관점에서의 국제경제학 이해"
vectorstore.similarity_search(query, k=2)

[Document(metadata={'author': 'Frederic S. Mishkin', 'isbn': 995171, 'pk': 455206329733087345, 'publisher': 'Pearson Education Ltd', 'title': 'The Economics of Money, Banking and Financial Markets'}, page_content='For courses in Money and Banking or General Economics.\n\nAn Analytical Framework for Understanding입니다.'),
 Document(metadata={'author': '남종현, 이홍식', 'isbn': 606380, 'pk': 455206329733087363, 'publisher': '경문사', 'title': '국제무역론: 이론과 정책'}, page_content='『국제무역론 이론과 정책』은 최근 국제무역환경의 급격한 변화에 따른 다양한 현상, 즉 아웃소싱, 직접투자, 서비스무역, 그리고 산업내 무역 등에 대한 현황과 특징을 상세히입니다.')]

In [ ]:
query = "미시경제와 거시경제의 비교"
vectorstore.similarity_search(query, k=2)

[Document(metadata={'author': '서승환', 'isbn': 751059, 'pk': 455206329733087336, 'publisher': '홍문사', 'title': '미시경제학'}, page_content='연세대학교 경제학부 교수 서승환의 『미시경제학』. 문장을 다듬어 최대한 평이하게 읽도록 직관적 설명으로 구성한 미시경제학 교재다. 행태경제학과 관련된 이론까지 다루고 있다'),
 Document(metadata={'author': 'Frederic S. Mishkin', 'isbn': 995171, 'pk': 455206329733087345, 'publisher': 'Pearson Education Ltd', 'title': 'The Economics of Money, Banking and Financial Markets'}, page_content='For courses in Money and Banking or General Economics.\n\nAn Analytical Framework for Understanding입니다.')]

## Milvus MMR 검색방식

In [ ]:
retriever = vectorstore.as_retriever(search_type="mmr", search_kwargs={"k": 2})
retriever.invoke("장기적 관점에서의 국제경제학 이해")

[Document(metadata={'author': '김신행, 김태기', 'isbn': 872787, 'pk': 455206329733087341, 'publisher': '법문사', 'title': '국제경제론'}, page_content='이 책은 국제경제론을 다룬 이론서입니다. 국제경제론의 기초적이고 전반적인 내용을 학습할 수 있도록 구성했습니다.'),
 Document(metadata={'author': '조반니 아리기', 'isbn': 951266, 'pk': 455206329733087250, 'publisher': '그린비', 'title': '장기 20세기'}, page_content='자본주의의 역사적 발전 및 순환 과정의 분석을 통해 미국 세계헤게모니 시기인 ‘장기 20세기’를 분석한 책. ‘장기 20세기’는 자본주의 세계체계의 특수한 발전 단계를 구성하는입니다.')]

In [ ]:
retriever = vectorstore.as_retriever(search_type="mmr", search_kwargs={"k": 2})
retriever.invoke("미시경제와 거시경제의 비교")

[Document(metadata={'author': '정운찬, 김영식', 'isbn': 599922, 'pk': 455206329733087339, 'publisher': '율곡출판사', 'title': '거시경제론'}, page_content='이 책은 거시경제론을 다룬 개론서입니다. 거시경제론의 기초적이고 전반적인 내용을 학습할 수 있도록 구성했습니다.'),
 Document(metadata={'author': 'David N. Weil', 'isbn': 156237, 'pk': 455206329733087344, 'publisher': 'Pearson College Div', 'title': 'Economic Growth'}, page_content='Why are some countries rich and others poor? David N. Weil, one of the top researchers in economic입니다.')]